In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

# Mount Drive, read file
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/aML/airbnb.csv')

Mounted at /content/drive


In [ ]:

# One-hot Encoding for city
df_encoded = pd.get_dummies(df, columns=['city'], prefix='city')

# Binary Encoding for time_period
df_encoded['is_weekend'] = (df_encoded['time_period'] == 'weekends').astype(int)

# Create combined stratification column
df_encoded['strat_column'] = df['city'] + '_' + df['time_period']

#Define features and target
X = df_encoded.drop(['price', 'log_price', 'time_period', 'strat_column'], axis=1)
y = df_encoded['log_price']

#Stratification split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_encoded['strat_column']
)

Lasso Regression


In [ ]:
# Pipeline for numeric features
numeric_cols = [
    'person_capacity',
    'cleanliness_rating',
    'guest_satisfaction_overall',
    'city_dist',
    'metro_dist'
]

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ],
    remainder='passthrough'
)

# Main pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('poly_features', PolynomialFeatures(interaction_only=True)),
    ('lasso_reg', Lasso())
])


param_grid = {
    'preprocessing__num__scaler': [StandardScaler(), 'passthrough'],
    'poly_features__degree': [1, 2, 3],
    'lasso_reg__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=3
)

grid_search.fit(X_train, y_train)

best_Lasso = grid_search.best_estimator_

y_pred = best_Lasso.predict(X_test)


score_train = best_Lasso.score(X_train, y_train)
score_test = best_Lasso.score(X_test, y_test)

print("Best parameters found:", grid_search.best_params_)
print("Best CV R2 score:", grid_search.best_score_)
print("Train R2 score:", score_train)
print("Test R2 score:", score_test)


lasso_step = best_Lasso.named_steps['lasso_reg']
coefficients = lasso_step.coef_
intercept = lasso_step.intercept_

print("\nIntercept:", intercept)

# Retrieve the PolynomialFeatures
poly_step = best_Lasso.named_steps['poly_features']

input_feature_names = X_train.columns

try:
    feature_names_poly = poly_step.get_feature_names_out(input_features=input_feature_names)
except AttributeError:
    feature_names_poly = poly_step.get_feature_names(input_feature_names)

coef_df = pd.DataFrame({
    'Feature': feature_names_poly,
    'Coefficient': coefficients
})
coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()

coef_df = coef_df.sort_values(by='AbsCoefficient', ascending=False)

print("\nCoefficients (sorted by absolute value):")
print(coef_df[['Feature', 'Coefficient']].head(20))

zeroed_features = coef_df[coef_df['Coefficient'] == 0.0]['Feature'].tolist()
print("\nFeatures pruned (coeff = 0):", zeroed_features)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.050e+03, tolerance: 1.457e+00
  model = cd_fast.enet_coordinate_descent(


Best parameters found: {'lasso_reg__alpha': 0.001, 'poly_features__degree': 3, 'preprocessing__num__scaler': 'passthrough'}
Best CV R2 score: 0.7146411952556415
Train R2 score: 0.7201537515651344
Test R2 score: 0.7245733597307253

Intercept: 5.77797471596428

Coefficients (sorted by absolute value):
                                            Feature  Coefficient
2                                      room_private    -0.087777
33                          room_shared entire_home     0.048641
923       host_is_superhost multipleRooms city_rome    -0.035864
642            room_private entire_home city_athens    -0.031017
417              room_shared metro_dist entire_home     0.029959
851           person_capacity is_luxury city_vienna     0.025409
57                         room_private city_athens    -0.023944
75                        person_capacity is_luxury     0.023586
432             room_shared entire_home city_athens    -0.019213
843           person_capacity is_luxury city_athe

Ridge Regression

In [ ]:
# Pipeline for numeric features
numeric_cols = [
    'person_capacity',
    'cleanliness_rating',
    'guest_satisfaction_overall',
    'city_dist',
    'metro_dist'
]

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ],
    remainder='passthrough'
)

# Main Ridge Pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('poly_features', PolynomialFeatures(interaction_only=True)),
    ('ridge_reg', Ridge())
])

param_grid = {
    'preprocessing__num__scaler': [StandardScaler(), 'passthrough'],
    'poly_features__degree': [1, 2, 3 ],
    'ridge_reg__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=3
)

grid_search.fit(X_train, y_train)
best_Ridge = grid_search.best_estimator_
y_pred = best_Ridge.predict(X)

print("Best parameters found:", grid_search.best_params_)
print("Best CV R2 score:", grid_search.best_score_)

score_train = best_Ridge.score(X_train, y_train)
score_test  = best_Ridge.score(X_test, y_test)

print("Train R2 score:", score_train)
print("Test R2 score:", score_test)

ridge_step = best_Ridge.named_steps['ridge_reg']
coefficients = ridge_step.coef_
intercept = ridge_step.intercept_

print("\nIntercept:", intercept)

# Retrieve the PolynomialFeatures step
poly_step = best_Ridge.named_steps['poly_features']

input_feature_names = X_train.columns

# Generate feature names from PolynomialFeatures
try:
    feature_names_poly = poly_step.get_feature_names_out(input_features=input_feature_names)
except AttributeError:

    feature_names_poly = poly_step.get_feature_names(input_feature_names)

coef_df = pd.DataFrame({
    'Feature': feature_names_poly,
    'Coefficient': coefficients
})
coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()

# Sort by absolute coefficient value to see most impactful
coef_df = coef_df.sort_values(by='AbsCoefficient', ascending=False)

print("\nCoefficients (sorted by absolute value):")
print(coef_df[['Feature', 'Coefficient']].head(20))  # show top 20

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters found: {'poly_features__degree': 3, 'preprocessing__num__scaler': 'passthrough', 'ridge_reg__alpha': 100}
Best CV R2 score: 0.7173446764963661
Train R2 score: 0.7290237449423469
Test R2 score: 0.7278271558671554

Intercept: 5.658853960152806

Coefficients (sorted by absolute value):
                                                Feature  Coefficient
1015         host_is_superhost is_luxury city_barcelona     0.103025
94                          host_is_superhost is_luxury     0.079765
37                           room_shared city_barcelona     0.068347
51              room_private guest_satisfaction_overall    -0.068277
445                   room_shared is_luxury city_berlin     0.064391
947      host_is_superhost cleanliness_rating is_luxury     0.061794
1503    guest_satisfaction_overall metro_dist is_luxury     0.059476
444                room_shared is_luxury city_barcelona    -0.057595
1016            h

In [ ]:
# Pipeline for numeric features
numeric_cols = [
    'person_capacity',
    'cleanliness_rating',
    'guest_satisfaction_overall',
    'city_dist',
    'metro_dist'
]

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ],
    remainder='passthrough'
)

# Main pipeline for Lasso
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('poly_features', PolynomialFeatures(interaction_only=False)),
    ('lasso_reg', Lasso())
])

param_grid = {
    'preprocessing__num__scaler': [StandardScaler(), 'passthrough'],
    'poly_features__degree': [1, 2, 3],
    'lasso_reg__alpha': [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=3
)

grid_search.fit(X_train, y_train)

best_Lasso = grid_search.best_estimator_

y_pred = best_Lasso.predict(X_test)


score_train = best_Lasso.score(X_train, y_train)
score_test = best_Lasso.score(X_test, y_test)

print("Best parameters found:", grid_search.best_params_)
print("Best CV R2 score:", grid_search.best_score_)
print("Train R2 score:", score_train)
print("Test R2 score:", score_test)


lasso_step = best_Lasso.named_steps['lasso_reg']
coefficients = lasso_step.coef_
intercept = lasso_step.intercept_

print("\nIntercept:", intercept)

# Retrieve the PolynomialFeatures step
poly_step = best_Lasso.named_steps['poly_features']

input_feature_names = X_train.columns

try:
    feature_names_poly = poly_step.get_feature_names_out(input_features=input_feature_names)
except AttributeError:
    feature_names_poly = poly_step.get_feature_names(input_feature_names)

coef_df = pd.DataFrame({
    'Feature': feature_names_poly,
    'Coefficient': coefficients
})
coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()

coef_df = coef_df.sort_values(by='AbsCoefficient', ascending=False)

print("\nCoefficients (sorted by absolute value):")
print(coef_df[['Feature', 'Coefficient']].head(20))

# Identify any features pruned to zero
zeroed_features = coef_df[coef_df['Coefficient'] == 0.0]['Feature'].tolist()
print("\nFeatures pruned (coeff = 0):", zeroed_features)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.603e+03, tolerance: 1.457e+00
  model = cd_fast.enet_coordinate_descent(


Best parameters found: {'lasso_reg__alpha': 0.0001, 'poly_features__degree': 3, 'preprocessing__num__scaler': StandardScaler()}
Best CV R2 score: 0.7268155398736186
Train R2 score: 0.7392097519945793
Test R2 score: 0.7399753600441923

Intercept: 5.062455865685763

Coefficients (sorted by absolute value):
                                              Feature  Coefficient
12                                          is_luxury     1.689823
223                          is_luxury city_amsterdam    -0.753211
14                                        city_athens    -0.750880
13                                     city_amsterdam     0.691146
232                             is_luxury city_vienna     0.652528
17                                      city_budapest    -0.504419
2165                entire_home is_luxury city_london    -0.436157
19                                        city_london     0.373955
230                              is_luxury city_paris    -0.355559
227                     

In [ ]:
# Pipeline for numeric features
numeric_cols = [
    'person_capacity',
    'cleanliness_rating',
    'guest_satisfaction_overall',
    'city_dist',
    'metro_dist'
]
numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ],
    remainder='passthrough'
)

# Main Ridge Pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('poly_features', PolynomialFeatures(interaction_only=False)),
    ('ridge_reg', Ridge())
])

param_grid = {
    'preprocessing__num__scaler': [StandardScaler(), 'passthrough'],
    'poly_features__degree': [1, 2, 3 ],
    'ridge_reg__alpha': [10, 50, 100, 250, 500, 1000]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=3
)

grid_search.fit(X_train, y_train)
best_Ridge = grid_search.best_estimator_
y_pred = best_Ridge.predict(X)

print("Best parameters found:", grid_search.best_params_)
print("Best CV R2 score:", grid_search.best_score_)

score_train = best_Ridge.score(X_train, y_train)
score_test  = best_Ridge.score(X_test, y_test)

print("Train R2 score:", score_train)
print("Test R2 score:", score_test)

# Extract the Ridge coefficients
ridge_step = best_Ridge.named_steps['ridge_reg']
coefficients = ridge_step.coef_
intercept = ridge_step.intercept_

print("\nIntercept:", intercept)

poly_step = best_Ridge.named_steps['poly_features']

input_feature_names = X_train.columns

try:
    feature_names_poly = poly_step.get_feature_names_out(input_features=input_feature_names)
except AttributeError:
    feature_names_poly = poly_step.get_feature_names(input_feature_names)

coef_df = pd.DataFrame({
    'Feature': feature_names_poly,
    'Coefficient': coefficients
})
coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()

coef_df = coef_df.sort_values(by='AbsCoefficient', ascending=False)

print("\nCoefficients (sorted by absolute value):")
print(coef_df[['Feature', 'Coefficient']].head(20))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters found: {'poly_features__degree': 3, 'preprocessing__num__scaler': 'passthrough', 'ridge_reg__alpha': 50}
Best CV R2 score: 0.7266350335601761
Train R2 score: 0.7412070588746339
Test R2 score: 0.737929568595233

Intercept: 4.265915255810159

Coefficients (sorted by absolute value):
                                              Feature  Coefficient
1195       host_is_superhost is_luxury city_barcelona     0.134850
98                        host_is_superhost is_luxury     0.110061
1192                    host_is_superhost is_luxury^2     0.110061
502                 room_shared is_luxury city_berlin     0.105995
1817  guest_satisfaction_overall metro_dist is_luxury     0.105420
501              room_shared is_luxury city_barcelona    -0.084461
1122   host_is_superhost cleanliness_rating is_luxury     0.082446
1089      host_is_superhost multipleRooms city_athens     0.080936
38                         room_share